In [1]:
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

try:
    import torchtext
except:
    !pip install torchtext
    
try:
    import spacy
except:
    !pip install spacy
    
try:
    spacy.load('en')
except:
    !python -m spacy download en
   

B:\Python\Python27\python.exe: No module named spacy


In [2]:
import torch
import torch.optim as optim
from torchbearer import Trial

import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(2,1)
        
    def forward(self, x):
        out = self.fc(x) 
        return out
      
model_HB = LinearRegression()
model_NAG = LinearRegression()

In [4]:
k = 2**4

w_ture = torch.rand(2,1)*5

# generate training set
pro = torch.distributions.multivariate_normal.MultivariateNormal(
    torch.Tensor([2,1]), covariance_matrix=torch.Tensor([[1,0],[0,1/k]]))
a = torch.zeros(500,2)
b = torch.zeros(500,1)
for i in range(500):
    a[i] = pro.sample()
    b[i] = a[i] @ w_ture   
# generate testing set
a_test = torch.zeros(500,2)
b_test = torch.zeros(500,1)
for i in range(500):
    a_test[i] = pro.sample()
    b_test[i] = a[i] @ w_ture 

In [5]:
def ave_loss_HB(lamada,m):
  
  losses = torch.Tensor([0])
  
  for i in range(100):
    nEpoch = 10
    model_HB = LinearRegression()
    optimizer = optim.SGD(model_HB.parameters(), lr=lamada, momentum=m)
    loss_func = nn.MSELoss()
    for epoch in range(nEpoch):
        for i in range(5): 
            optimizer.zero_grad()
            b_hat = model_HB(a)
            loss = loss_func(b_hat,b)
            loss.backward()
            optimizer.step()
        
    b_hat_test = model_HB(a_test)
    loss_func_test = nn.MSELoss()
    losses = losses + loss_func_test(b_test,b_hat_test)
    losses = losses/100
  
    return losses

In [6]:
def ave_loss_NAG(lamada,m):
    losses = torch.Tensor([0])
    for i in range(100):
        nEpoch = 10
        model_NAG = LinearRegression()
        optimizer = optim.SGD(model_HB.parameters(), lr=lamada, momentum=m, weight_decay=0, dampening=0, nesterov=True)
        loss_func = nn.MSELoss()
        for epoch in range(nEpoch):
            for i in range(5): 
                optimizer.zero_grad()
                b_hat = model_NAG(a)
                loss = loss_func(b_hat,b)
                loss.backward()
                optimizer.step()
    b_hat_test = model_NAG(a_test)
    loss_func_test = nn.MSELoss()
    losses = losses + loss_func_test(b_test,b_hat_test)
    losses = losses/100
  
    return losses

In [1]:
lm_set = np.linspace(0.1,1,10)
m_set = np.linspace(0.1,1,10)
loss_set_HB = np.zeros([10,10])

x = 0
for i in lm_set:
    y = 0
    for j in m_set:
        loss = ave_loss_HB(i,j)
        loss_set_HB[x,y] = loss
        #print(x,y)
        y += 1
    x += 1
print(loss_set_HB)
np.min(loss_set_HB)

In [2]:
lm_set = np.linspace(0.1,1,10)
m_set = np.linspace(0.1,1,10)
loss_set_NAG = np.zeros([10,10])

x = 0
for i in lm_set:
    y = 0
    for j in m_set:
        loss = ave_loss_NAG(i,j)
        loss_set_NAG[x,y] = loss
        #print(x,y)
        y += 1
    x += 1
print(loss_set_NAG)
np.min(loss_set_NAG)